# Importando as bibliotecas

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.model_selection import GroupKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# Carregando os dados

In [3]:

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


# Dividindo os dados de treino e teste

In [4]:
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


# Modelo DummyClassifier(bobo) para gerar uma 'nota' baseline

In [5]:
dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)

A acurácia do dummy stratified foi de 50.96%


c:\python38\lib\site-packages\sklearn\dummy.py:131: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  warnings.warn("The default value of strategy will change from "


# Modelo de árvode de descisão para classificação

In [6]:
SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


## Estratificando os dados

In [7]:
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia do dummy stratified foi 71.92%


## Modificando o SEED

In [8]:
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia do dummy stratified foi 76.84%


## Utilizando validação cruzada

In [9]:
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x,y, cv=5, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print('Acurácia com cross validation, 5 = [%.2f,%.2f]' % ((media-2*desvio_padrao)*100, (media+2*desvio_padrao)*100))

Acurácia com cross validation, 5 = [75.21,76.35]


# Aleatoriedade no cross validate 

In [10]:
def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Acurácia média %.2f "%(media*100))
    print('Acurácia com cross validation, 5 = [%.2f,%.2f]' % ((media-2*desvio_padrao)*100, (media+2*desvio_padrao)*100))

In [11]:
cv = KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x,y, cv=cv, return_train_score=False)
imprime_resultados(results)

Acurácia média 75.77 
Acurácia com cross validation, 5 = [73.35,78.19]


# Simular situação horrivel ou azar
Pode ser 'azar' como pode ser uma proporção de exemplos dsbalanceado entre as classes

In [12]:
dados_com_azar = dados.sort_values('vendido', ascending=True)
x_azar = dados_com_azar[['preco','idade_do_modelo','km_por_ano']]
y_azar = dados_com_azar['vendido']

In [13]:
cv = KFold(n_splits=10, shuffle=False)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar,y_azar, cv=cv, return_train_score=False)
imprime_resultados(results)

Acurácia média 57.84 
Acurácia com cross validation, 5 = [34.29,81.39]


In [14]:
cv = KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar,y_azar, cv=cv, return_train_score=False)
imprime_resultados(results)

Acurácia média 75.78 
Acurácia com cross validation, 5 = [73.80,77.76]


In [15]:
cv = StratifiedKFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar,y_azar, cv=cv, return_train_score=False)
imprime_resultados(results)

Acurácia média 75.78 
Acurácia com cross validation, 5 = [74.23,77.33]


# Gerando dados aleatório de modelo de carro para simulção de agruapameto ao usar nosso estimador

In [16]:
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2,3,size=10000)
dados['modelo'] = dados['modelo'] + abs(dados['modelo'].min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,21
1,40557.96,1,20,12622.05362,20
2,89627.50,0,12,11440.79806,13
3,95276.14,0,3,43167.32682,3
4,117384.68,1,4,12770.11290,8


In [17]:
dados.modelo.value_counts()

20    852
21    796
19    763
18    737
17    703
16    699
14    612
22    611
15    597
13    542
12    463
11    450
10    367
23    366
9     313
8     277
7     245
24    198
6     151
5     116
4      72
3      48
2      18
1       4
Name: modelo, dtype: int64

# Testando validação cruzada com GroupKFold

In [18]:
cv = GroupKFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar,y_azar, cv=cv,groups=dados.modelo, return_train_score=False)
imprime_resultados(results)

Acurácia média 75.79 
Acurácia com cross validation, 5 = [73.16,78.41]


# Cross validation com StandardScaler

In [19]:
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes)*100
print('A acurácia foi %.2f%%'%acuracia)

A acurácia foi 58.00%


In [20]:
cv = GroupKFold(n_splits=10)
modelo = SVC()
results = cross_validate(modelo, x_azar,y_azar, cv=cv,groups=dados.modelo, return_train_score=False)
imprime_resultados(results)

Acurácia média 77.24 
Acurácia com cross validation, 5 = [74.16,80.33]


fit() é sempre feito com o dados de treino

## Pipeline

In [21]:
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformação',scaler), ('estimador',modelo)])
pipeline

Pipeline(steps=[('transformação', StandardScaler()), ('estimador', SVC())])

In [22]:
cv = GroupKFold(n_splits=10)
results = cross_validate(pipeline, x_azar,y_azar, cv=cv,groups=dados.modelo, return_train_score=False)
imprime_resultados(results)

Acurácia média 76.65 
Acurácia com cross validation, 5 = [74.07,79.22]
